[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/corazzon/finance-data-analysis/blob/main/3.3%20%EB%84%A4%EC%9D%B4%EB%B2%84%EA%B8%88%EC%9C%B5%20%EA%B0%9C%EB%B3%84%EC%A2%85%EB%AA%A9%20%EC%88%98%EC%A7%91-input.ipynb)


## 네이버 금융 개별종목 수집
* FinanceDataReader를 통해 수집했던 데이터를 네이버 증권 웹 페이지를 통해 직접 수집합니다.


### Keyword

* html 파일 읽어오기
    * pd.read_html(url, encoding="cp949")

* 결측 데이터 제거하기(axis 0:행, 1:열)
    * table[0].dropna()

* 데이터 프레임 합치기
    * pd.concat([df1, df2, df3])

* 중복데이터 제거
    * df.drop_duplicates()

* 과학적 기수법
    * 1.210000e+02 => 121

* 날짜 column의 첫 row값 확인
    * date = df.iloc[0]["날짜"]

* 파일로 저장하기 
    * df.to_csv(file_name, index=False)

* 파일 읽어오기
    * pd.read_csv(file_name)

## 수집할 페이지 보기

* 네이버 금융 국내증시 : https://finance.naver.com/sise/
* 2020년 주요 상장종목
    * 빅히트 : https://finance.naver.com/item/main.nhn?code=352820
    * 카카오게임즈 : https://finance.naver.com/item/main.nhn?code=293490
    * SK바이오팜 : https://finance.naver.com/item/main.nhn?code=326030

## 라이브러리 로드

In [1]:
# 라이브러리 로드
import pandas as pd

## pandas 코드 한 줄로 데이터 수집하기

In [5]:
# read_html을 이용하여 url의 page내의 값을 DataFrame으로 받아옵니다.
# cp949는 한글 인코딩을 위해 사용합니다. 기본 인코딩 설정은 utf-8 이며, 
# 네이버의 일별 시세는 cp949 인코딩으로 불러올 수 있습니다.
# 데이터를 로드 했을 때 한글 인코딩이 깨진다면 대부분 cp949 로 불러올 수 있습니다.


url = "https://finance.naver.com/item/sise.naver?code=005930"
table = pd.read_html(url, encoding='CP949') 
len(table) # 일별 시세를 보기 위해 총 12개의 Table(표)를 불러왔다 

12

In [6]:
# table[0]와 table[1]을 확인하여 보면 table[0]에 필요한 데이터들이 있습니다.
table[0] # [0]~[11]: 일별시세는 찾을 수 없었다

,0,1,2
0,"전일 70,600 70,600","고가 71,00071,000 (상한가 91,70091,700 )","거래량 3,807,312 3,807,312"
1,"시가 70,90070,900","저가 70,20070,200 (하한가 49,500 )","거래대금 268,937 268,937 백만"


## 수집할 URL 정하기

In [2]:
# 종목번호와 상장사 이름을 item_code와 item_name으로 설정
# item_code = "352820"
# item_name = "빅히트"

# item_code = "326030"
# item_name = "SK바이오팜"

# 종목 URL 만들기
url = "https://finance.naver.com/item/sise_day.naver?code=005930&page=5"


## requests를 통한 HTTP 요청
* [Requests: HTTP for Humans™ — Requests documentation](https://requests.readthedocs.io/en/master/)
* [Quickstart — Requests documentation # custom-headers](https://requests.readthedocs.io/en/latest/user/quickstart/#custom-headers)

In [3]:
import requests

headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36'}

response = requests.get(url, headers=headers) # <Response [200]> : 정상적으로 잘 받았다라는 의미
response.text

'\n<html lang="ko">\n<head>\n<meta http-equiv="Content-Type" content="text/html; charset=euc-kr">\n<title>네이버 증권</title>\n\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20230728132835/css/newstock.css">\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20230728132835/css/common.css">\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20230728132835/css/layout.css">\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20230728132835/css/main.css">\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20230728132835/css/newstock2.css">\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20230728132835/css/newstock3.css">\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20230728132835/css/world.css">\n</head>\n<body>\n<script

## BeautifulSoup 을 통한 table 태그 찾기

* [Beautiful Soup Documentation — Beautiful Soup 4.9.0 documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)

In [14]:
from bs4 import BeautifulSoup as bs
# bs(response.text, "lxml") # 에러가 뜨게 되면 "lxml" 넣기 
html = bs(response.text)
table = html.select("table")

In [18]:
table[0]

<table cellspacing="0" class="type2">
<tr>
<th>날짜</th>
<th>종가</th>
<th>전일비</th>
<th>시가</th>
<th>고가</th>
<th>저가</th>
<th>거래량</th>
</tr>
<tr>
<td colspan="7" height="8"></td>
</tr>
<tr onmouseout="mouseOut(this)" onmouseover="mouseOver(this)">
<td align="center"><span class="tah p10 gray03">2023.06.02</span></td>
<td class="num"><span class="tah p11">72,200</span></td>
<td class="num">
<img alt="상승" height="6" src="https://ssl.pstatic.net/imgstock/images/images4/ico_up.gif" style="margin-right:4px;" width="7"/><span class="tah p11 red02">
				1,300
				</span>
</td>
<td class="num"><span class="tah p11">71,700</span></td>
<td class="num"><span class="tah p11">72,200</span></td>
<td class="num"><span class="tah p11">71,600</span></td>
<td class="num"><span class="tah p11">12,161,798</span></td>
</tr>
<tr onmouseout="mouseOut(this)" onmouseover="mouseOver(this)">
<td align="center"><span class="tah p10 gray03">2023.06.01</span></td>
<td class="num"><span class="tah p11">70,900</span></td>


## pandas 코드 한 줄로 데이터 수집하기

In [20]:
# read_html을 이용하여 url의 page내의 값을 DataFrame으로 받아옵니다.
# cp949는 한글 인코딩을 위해 사용합니다. 기본 인코딩 설정은 utf-8 이며, 
# 네이버의 일별 시세는 cp949 인코딩으로 불러올 수 있습니다.
# 데이터를 로드 했을 때 한글 인코딩이 깨진다면 대부분 cp949 로 불러올 수 있습니다.

pd.read_html(str(table))[0]

,날짜,종가,전일비,시가,고가,저가,거래량
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023.06.02,72200.0,1300.0,71700.0,72200.0,71600.0,12161798.0
2,2023.06.01,70900.0,500.0,70900.0,71600.0,70600.0,14669296.0
3,2023.05.31,71400.0,900.0,72400.0,72500.0,71000.0,25666087.0
4,2023.05.30,72300.0,2000.0,71300.0,72300.0,71200.0,27476897.0
5,2023.05.26,70300.0,1500.0,69800.0,70400.0,69500.0,19549511.0
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2023.05.25,68800.0,300.0,69900.0,70000.0,68700.0,14231160.0


In [21]:
# table[0]와 table[1]을 확인하여 보면 table[0]에 필요한 데이터들이 있습니다.
table[0]

<table cellspacing="0" class="type2">
<tr>
<th>날짜</th>
<th>종가</th>
<th>전일비</th>
<th>시가</th>
<th>고가</th>
<th>저가</th>
<th>거래량</th>
</tr>
<tr>
<td colspan="7" height="8"></td>
</tr>
<tr onmouseout="mouseOut(this)" onmouseover="mouseOver(this)">
<td align="center"><span class="tah p10 gray03">2023.06.02</span></td>
<td class="num"><span class="tah p11">72,200</span></td>
<td class="num">
<img alt="상승" height="6" src="https://ssl.pstatic.net/imgstock/images/images4/ico_up.gif" style="margin-right:4px;" width="7"/><span class="tah p11 red02">
				1,300
				</span>
</td>
<td class="num"><span class="tah p11">71,700</span></td>
<td class="num"><span class="tah p11">72,200</span></td>
<td class="num"><span class="tah p11">71,600</span></td>
<td class="num"><span class="tah p11">12,161,798</span></td>
</tr>
<tr onmouseout="mouseOut(this)" onmouseover="mouseOver(this)">
<td align="center"><span class="tah p10 gray03">2023.06.01</span></td>
<td class="num"><span class="tah p11">70,900</span></td>


In [23]:
# dropna를 통해 결측치가 들어있는 row를 제거합니다.
temp = pd.read_html(str(table))[0]
temp.dropna() # 결측치있는 행 모두 삭제

,날짜,종가,전일비,시가,고가,저가,거래량
1,2023.06.02,72200.0,1300.0,71700.0,72200.0,71600.0,12161798.0
2,2023.06.01,70900.0,500.0,70900.0,71600.0,70600.0,14669296.0
3,2023.05.31,71400.0,900.0,72400.0,72500.0,71000.0,25666087.0
4,2023.05.30,72300.0,2000.0,71300.0,72300.0,71200.0,27476897.0
5,2023.05.26,70300.0,1500.0,69800.0,70400.0,69500.0,19549511.0
9,2023.05.25,68800.0,300.0,69900.0,70000.0,68700.0,14231160.0
10,2023.05.24,68500.0,100.0,68100.0,68700.0,68000.0,8192896.0
11,2023.05.23,68400.0,100.0,68500.0,68700.0,68100.0,8561643.0
12,2023.05.22,68500.0,100.0,68400.0,69000.0,68000.0,14470308.0
13,2023.05.19,68400.0,2200.0,67800.0,68400.0,67500.0,20349345.0


## 수집할 URL 정하기

In [25]:
# 종목번호와 상장사 이름을 item_code와 item_name으로 설정
# item_code = "352820"
# item_name = "빅히트"

item_code = "326030"
item_name = "SK바이오팜"
page_no = 1

# 종목 URL 만들기
url = f"https://finance.naver.com/item/sise_day.naver?code={item_code}&page={page_no}"
url

'https://finance.naver.com/item/sise_day.naver?code=326030&page=1'

## 페이지별 데이터 수집 함수 만들기

In [38]:
# 종목 번호를 이용해 page에 따라 데이터를 읽어오는 함수
# """ 는 이 두개 사이의 행들은 주석 처리되며, 함수의 docstring 으로 사용됩니다.

def get_day_list(item_code, page_no):
    """
    일자별 시세를 페이지별로 수집
    """ 
    url = f"https://finance.naver.com/item/sise_day.naver?code={item_code}&page={page_no}"
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36'}

    response = requests.get(url, headers=headers) # <Response [200]> : 정상적으로 잘 받았다라는 의미
    html = bs(response.text)
    table = html.select('table')
    temp = pd.read_html(str(table))
    temp = temp[0].dropna()
    return temp

In [44]:
# 함수가 잘 만들어졌는지 확인하기
get_day_list(item_code, page_no)
# get_day_list(item_code, 2)
# get_day_list(item_code, 30)

,날짜,종가,전일비,시가,고가,저가,거래량
1,2022.05.30,87900.0,200.0,88500.0,88500.0,87600.0,65884.0
2,2022.05.27,87700.0,0.0,87800.0,89200.0,87000.0,95553.0
3,2022.05.26,87700.0,300.0,87500.0,88900.0,87000.0,89766.0
4,2022.05.25,87400.0,1900.0,86400.0,88300.0,85700.0,133473.0
5,2022.05.24,85500.0,1100.0,86600.0,87900.0,85100.0,85118.0
9,2022.05.23,86600.0,300.0,86900.0,87900.0,85900.0,80747.0
10,2022.05.20,86300.0,300.0,86600.0,86800.0,85600.0,77180.0
11,2022.05.19,86000.0,2100.0,87000.0,87400.0,85700.0,100423.0
12,2022.05.18,88100.0,400.0,87600.0,88700.0,86800.0,81872.0
13,2022.05.17,88500.0,200.0,88700.0,88900.0,87700.0,91205.0


## 반복문을 통한 전체 일자 데이터 수집하기
* (주의) 기간이 긴 데이터를 수집할때는 서버에 부담을 주지 않기 위해 time.sleep()값을 주세요.

In [57]:
import time
# web page 시작번호
page_no = 1
# 데이터를 저장할 빈 변수 선언
item_list = []

while True:
    print(page_no)
    temp = get_day_list(item_code, page_no)
    item_list.append(temp)
    page_no += 1
    time.sleep(0.1) # 일방적으로 데이터를 불러오면 디도스 공격으로 오인받을 수 있음
    if temp.shape[0] < 10:
        break
    elif page_no > 10:
        break
        

1
2
3
4
5
6
7
8
9
10


In [58]:
item_list

[            날짜       종가     전일비       시가       고가       저가       거래량
 1   2023.07.31  81400.0   300.0  81700.0  83200.0  81000.0   51252.0
 2   2023.07.28  81700.0  2300.0  83200.0  84000.0  80700.0  122801.0
 3   2023.07.27  84000.0     0.0  83400.0  84900.0  82800.0  177218.0
 4   2023.07.26  84000.0  2300.0  81700.0  84500.0  79500.0  313396.0
 5   2023.07.25  81700.0  2900.0  84300.0  85500.0  81500.0  149965.0
 9   2023.07.24  84600.0   800.0  85700.0  86700.0  83300.0  176847.0
 10  2023.07.21  85400.0     0.0  85400.0  87000.0  84000.0  233863.0
 11  2023.07.20  85400.0  3800.0  81600.0  87000.0  81100.0  380258.0
 12  2023.07.19  81600.0  6200.0  75600.0  82300.0  74700.0  278945.0
 13  2023.07.18  75400.0  1000.0  76400.0  77100.0  74700.0   90659.0,
             날짜       종가     전일비       시가       고가       저가       거래량
 1   2023.07.17  76400.0   100.0  77600.0  78000.0  75900.0   86559.0
 2   2023.07.14  76500.0   800.0  76500.0  77800.0  75900.0  128510.0
 3   2023.07.13  75

## 수집한 데이터 하나의 데이터프레임으로 합치기

<img src="https://pandas.pydata.org/docs/_images/merging_concat_basic.png">

* [Merge, join, concatenate and compare documentation](https://pandas.pydata.org/docs/user_guide/merging.html#merge-join-concatenate-and-compare)

In [60]:
# DataFrame들이 list의 형태로 저장된 것을 concat을 이용하여 합치면서 하나의 DataFrame으로 만듭니다.
df = pd.concat(item_list) # 컬럼명이 일치하면 합쳐짐

<img src="https://pandas.pydata.org/docs/_images/08_concat_row.svg">

In [61]:
# head와 tail로 데이터의 일부를 가져와서 봅니다.
df.head()

,날짜,종가,전일비,시가,고가,저가,거래량
1,2023.07.31,81400.0,300.0,81700.0,83200.0,81000.0,51252.0
2,2023.07.28,81700.0,2300.0,83200.0,84000.0,80700.0,122801.0
3,2023.07.27,84000.0,0.0,83400.0,84900.0,82800.0,177218.0
4,2023.07.26,84000.0,2300.0,81700.0,84500.0,79500.0,313396.0
5,2023.07.25,81700.0,2900.0,84300.0,85500.0,81500.0,149965.0


In [62]:
df.tail()

,날짜,종가,전일비,시가,고가,저가,거래량
9,2023.03.14,60600.0,700.0,60900.0,61700.0,59800.0,142129.0
10,2023.03.13,61300.0,1100.0,62100.0,62600.0,60500.0,102724.0
11,2023.03.10,62400.0,500.0,62300.0,62900.0,61200.0,90882.0
12,2023.03.09,62900.0,200.0,62700.0,63800.0,62100.0,115872.0
13,2023.03.08,62700.0,3000.0,65100.0,65100.0,62400.0,172594.0


## 데이터프레임에 종목코드와 종목명을 추가하기
* 파생변수 만들기

In [64]:
# '종목코드'와 '종목명' column을 추가하면서 각각 item_code와 item_name 값을 입력합니다.
df["종목코드"] = item_code
df["종목명"] = item_name
df

,날짜,종가,전일비,시가,고가,저가,거래량,종목코드,종목명
1,2023.07.31,81400.0,300.0,81700.0,83200.0,81000.0,51252.0,326030,SK바이오팜
2,2023.07.28,81700.0,2300.0,83200.0,84000.0,80700.0,122801.0,326030,SK바이오팜
3,2023.07.27,84000.0,0.0,83400.0,84900.0,82800.0,177218.0,326030,SK바이오팜
4,2023.07.26,84000.0,2300.0,81700.0,84500.0,79500.0,313396.0,326030,SK바이오팜
5,2023.07.25,81700.0,2900.0,84300.0,85500.0,81500.0,149965.0,326030,SK바이오팜
...,...,...,...,...,...,...,...,...,...
9,2023.03.14,60600.0,700.0,60900.0,61700.0,59800.0,142129.0,326030,SK바이오팜
10,2023.03.13,61300.0,1100.0,62100.0,62600.0,60500.0,102724.0,326030,SK바이오팜
11,2023.03.10,62400.0,500.0,62300.0,62900.0,61200.0,90882.0,326030,SK바이오팜
12,2023.03.09,62900.0,200.0,62700.0,63800.0,62100.0,115872.0,326030,SK바이오팜


## 컬럼 순서 변경하기

In [66]:
# DataFrame에서 column 들의 이름을 순서를 조정하여 column순서를 변경할 수 있습니다.
df.columns

Index(['날짜', '종가', '전일비', '시가', '고가', '저가', '거래량', '종목코드', '종목명'], dtype='object')

In [68]:
cols = ['종목코드', '종목명', '날짜', '종가', '전일비', '시가', '고가', '저가', '거래량']
df = df[cols]

## 중복데이터 제거하기
* drop_duplicates 를 통해 중복된 데이터가 있다면 제거합니다.

In [70]:
# drop_duplicates : row들 끼리 data를 비교하여 같은 값이 있으면 row중 하나를 삭제
# df.shape를 전/후로 출력하여 삭제된 row가 있는지 확인
df = df.drop_duplicates()

## 기술통계값 구하기

In [72]:
# describe는 기본적으로 수치데이터에 대한 기술통계값을 구하게 됩니다.
df.describe()

,종가,전일비,시가,고가,저가,거래량
count,100.000000,100.00000,100.000000,100.000000,100.000000,100.000000
mean,72608.000000,1369.00000,72459.000000,73919.000000,71302.000000,147194.130000
std,7761.204738,1376.98947,7772.550704,7970.180473,7520.436534,96051.237416
min,59000.000000,0.00000,58800.000000,59900.000000,57600.000000,41428.000000
25%,68150.000000,500.00000,67275.000000,69050.000000,65925.000000,90208.000000
50%,72400.000000,1000.00000,72150.000000,73750.000000,71350.000000,120814.500000
75%,79050.000000,1625.00000,77975.000000,81550.000000,77025.000000,173212.750000
max,85900.000000,6200.00000,86200.000000,87300.000000,84700.000000,693322.000000


* 참고 : E notation
[과학적 기수법 - 위키백과, 우리 모두의 백과사전](https://ko.wikipedia.org/wiki/%EA%B3%BC%ED%95%99%EC%A0%81_%EA%B8%B0%EC%88%98%EB%B2%95)

과학적 기수법, 과학적 표기법(scientific notation, scientific form, standard index form, standard form)은 너무 크거나 너무 작은 숫자들을 십진법으로 편하게 작성하여 표현하는 방법이다. 과학자, 수학자, 공학자들이 공통적으로 사용하는데, 부분적인 이유는 특정한 산술을 단순화시켜 주기 때문이다. 과학 계산기에서는 "SCI" 디스플레이 모드라는 이름으로 알려져 있다.

In [73]:
# 거래량의 과학적 기수법 읽기
1.0000000e+02

100.0

## 최근 날짜 구해서 파일명 만들기

In [76]:
# 날짜 column의 첫 row값 확인
date = df.iloc[0]["날짜"]
date

'2023.07.31'

In [80]:
# 종목명, 종목코드, 날짜를 이름으로 하는 csv 파일명 만들기
file_name = f"{item_name}_{item_code}_{date}.csv"
file_name

'SK바이오팜_326030_2023.07.31.csv'

## 파일로 저장하기

In [81]:
# 파일로 저장하기 
# index=False 로 데이터프레임의 기본 index 는 저장하지 않도록 합니다.
df.to_csv(file_name, index=False)

In [83]:
# 제대로 저장되었는지 파일을 읽어서 확인합니다.
pd.read_csv(file_name)

,종목코드,종목명,날짜,종가,전일비,시가,고가,저가,거래량
0,326030,SK바이오팜,2023.07.31,81400.0,300.0,81700.0,83200.0,81000.0,51252.0
1,326030,SK바이오팜,2023.07.28,81700.0,2300.0,83200.0,84000.0,80700.0,122801.0
2,326030,SK바이오팜,2023.07.27,84000.0,0.0,83400.0,84900.0,82800.0,177218.0
3,326030,SK바이오팜,2023.07.26,84000.0,2300.0,81700.0,84500.0,79500.0,313396.0
4,326030,SK바이오팜,2023.07.25,81700.0,2900.0,84300.0,85500.0,81500.0,149965.0
...,...,...,...,...,...,...,...,...,...
95,326030,SK바이오팜,2023.03.14,60600.0,700.0,60900.0,61700.0,59800.0,142129.0
96,326030,SK바이오팜,2023.03.13,61300.0,1100.0,62100.0,62600.0,60500.0,102724.0
97,326030,SK바이오팜,2023.03.10,62400.0,500.0,62300.0,62900.0,61200.0,90882.0
98,326030,SK바이오팜,2023.03.09,62900.0,200.0,62700.0,63800.0,62100.0,115872.0


## 전체 과정을 하나의 함수로 만들기

In [84]:
def get_item_list(item_code, item_name):
    """
    일별 시세를 수집하는 함수
    """
    page_no = 1
    # 데이터를 저장할 빈 변수 선언
    item_list = []

    while True:
        print(page_no)
        temp = get_day_list(item_code, page_no)
        item_list.append(temp)
        page_no += 1
        time.sleep(0.1) # 일방적으로 데이터를 불러오면 디도스 공격으로 오인받을 수 있음
        if page_no > 10:
            break
    df = pd.concat(item_list)
    df["종목코드"] = item_code
    df["종목명"] = item_name
    cols = ['종목코드', '종목명', '날짜', '종가', '전일비', '시가', '고가', '저가', '거래량']
    df = df[cols]
    return df



In [87]:
item_code = "352820"
item_name = "하이브"
# 빅히트의 사명이 하이브로 변경되었습니다.

# item_code = "326030"
# item_name = "SK바이오팜"
get_item_list(item_code,item_name)

1
2
3
4
5
6
7
8
9
10


,종목코드,종목명,날짜,종가,전일비,시가,고가,저가,거래량
1,352820,하이브,2023.07.31,261000.0,2500.0,259500.0,264000.0,258000.0,134377.0
2,352820,하이브,2023.07.28,258500.0,3000.0,259000.0,264500.0,254000.0,183611.0
3,352820,하이브,2023.07.27,261500.0,4500.0,258000.0,266500.0,258000.0,264353.0
4,352820,하이브,2023.07.26,257000.0,14000.0,274500.0,274500.0,257000.0,403873.0
5,352820,하이브,2023.07.25,271000.0,9000.0,264000.0,278500.0,262000.0,516976.0
...,...,...,...,...,...,...,...,...,...
9,352820,하이브,2023.03.14,173700.0,15900.0,187700.0,189300.0,173000.0,431611.0
10,352820,하이브,2023.03.13,189600.0,5900.0,185600.0,196600.0,182600.0,849993.0
11,352820,하이브,2023.03.10,183700.0,7200.0,174100.0,189800.0,173900.0,505221.0
12,352820,하이브,2023.03.09,176500.0,700.0,177800.0,179500.0,174800.0,216070.0


## 내가 만든 함수의 기능과 소스코드가 궁금하다면?

In [88]:
# 도움말 보기
# get_item_list?


In [ ]:
# 소스코드 보기
# get_item_list??
